<a href="https://colab.research.google.com/github/stav1236/google-machine-learning-course/blob/main/foundational-courses/2.machine-learning-crash-course/ml-models/classification/classification-credit-card-score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# Import required libraries
import io
import keras
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import plotly.express as px
import dataclasses
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
dataset_url = "https://raw.githubusercontent.com/stav1236/google-machine-learning-course/main/foundational-courses/2.machine-learning-crash-course/ml-models/classification/datasets/Score.csv"  # Update this path with the actual file
dataset = pd.read_csv(dataset_url)

@dataclasses.dataclass()
class ExperimentSettings:
  learning_rate: float
  number_epochs: int
  batch_size: int
  classification_threshold: float
  input_features: list[str]

@dataclasses.dataclass()
class Experiment:
  name: str
  settings: ExperimentSettings
  model: keras.Model
  epochs: np.ndarray
  metrics_history: keras.callbacks.History

def create_model(settings: ExperimentSettings, metrics: list[keras.metrics.Metric]) -> keras.Model:
  model_inputs = [keras.Input(name=feature, shape=(1,)) for feature in settings.input_features]
  concatenated_inputs = keras.layers.Concatenate()(model_inputs)
  dense = keras.layers.Dense(units=1, input_shape=(1,), activation=keras.activations.sigmoid)
  model_output = dense(concatenated_inputs)
  model = keras.Model(inputs=model_inputs, outputs=model_output)
  model.compile(
      optimizer=keras.optimizers.RMSprop(settings.learning_rate),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics,
  )
  return model

def train_model(experiment_name: str, model: keras.Model, dataset: pd.DataFrame, labels: np.ndarray, settings: ExperimentSettings) -> Experiment:
  features = {feature_name: np.array(dataset[feature_name]) for feature_name in settings.input_features}
  history = model.fit(x=features, y=labels, batch_size=settings.batch_size, epochs=settings.number_epochs)
  return Experiment(experiment_name, settings, model, history.epoch, pd.DataFrame(history.history))

# Evaluation
def evaluate_experiment(experiment: Experiment, test_dataset: pd.DataFrame, test_labels: np.array) -> dict[str, float]:
  features = {feature_name: np.array(test_dataset[feature_name]) for feature_name in experiment.settings.input_features}
  return experiment.model.evaluate(x=features, y=test_labels, batch_size=experiment.settings.batch_size, verbose=0, return_dict=True)

# Define experiment settings
settings = ExperimentSettings(
    learning_rate=0.001,
    number_epochs=60,
    batch_size=100,
    classification_threshold=0.35,
    input_features = [
    'Delay_from_due_date',
    'Num_of_Delayed_Payment',
    'Num_Credit_Inquiries',
    'Credit_Utilization_Ratio',
    'Credit_History_Age',
    'Amount_invested_monthly',
    'Monthly_Balance',
    'Age',
    'Annual_Income',
    'Num_Bank_Accounts',
    'Num_Credit_Card',
    'Interest_Rate',
    'Num_of_Loan',
    'Monthly_Inhand_Salary',
    'Changed_Credit_Limit',
    'Outstanding_Debt',
    'Total_EMI_per_month'
    ]
)

metrics = [
    keras.metrics.BinaryAccuracy(name='accuracy', threshold=settings.classification_threshold),
    keras.metrics.Precision(name='precision', thresholds=settings.classification_threshold),
    keras.metrics.Recall(name='recall', thresholds=settings.classification_threshold),
    keras.metrics.AUC(num_thresholds=100, name='auc'),
]

# Normalization
feature_mean = dataset.mean(numeric_only=True)
feature_std = dataset.std(numeric_only=True)
numerical_features = dataset.select_dtypes('number').columns
normalized_dataset = (dataset[numerical_features] - feature_mean) / feature_std
normalized_dataset['Is_Poor'] = (dataset['Credit_Score'] == "Poor")
normalized_dataset['Is_Standard'] = (dataset['Credit_Score'] == "Standard")
normalized_dataset['Is_Good'] = (dataset['Credit_Score'] == "Good")

# Splitting the dataset
number_samples = len(normalized_dataset)
index_80th = round(number_samples * 0.8)

shuffled_dataset = normalized_dataset.sample(frac=1, random_state=100)
train_data = shuffled_dataset.iloc[0:index_80th]
test_data = shuffled_dataset.iloc[index_80th:]

poor_vs_all_model = create_model(settings, metrics)
standard_vs_all_model = create_model(settings, metrics)
good_vs_all_model = create_model(settings, metrics)


experiment_poor_vs_all = train_model('baseline', poor_vs_all_model, train_data.drop(columns=['Is_Poor','Is_Standard','Is_Good']), train_data['Is_Poor'].to_numpy(), settings)
test_metrics_poor_vs_all = evaluate_experiment(experiment_poor_vs_all, test_data.drop(columns=['Is_Poor','Is_Standard','Is_Good']), test_data['Is_Poor'].to_numpy())

experiment_standard_vs_all = train_model('baseline', standard_vs_all_model, train_data.drop(columns=['Is_Poor','Is_Standard','Is_Good']), train_data['Is_Standard'].to_numpy(), settings)
test_metrics_standard_vs_all = evaluate_experiment(experiment_standard_vs_all, test_data.drop(columns=['Is_Poor','Is_Standard','Is_Good']), test_data['Is_Standard'].to_numpy())

experiment_good_vs_all = train_model('baseline', good_vs_all_model, train_data.drop(columns=['Is_Poor','Is_Standard','Is_Good']), train_data['Is_Good'].to_numpy(), settings)
test_metrics_good_vs_all = evaluate_experiment(experiment_good_vs_all, test_data.drop(columns=['Is_Poor','Is_Standard','Is_Good']), test_data['Is_Good'].to_numpy())


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/60
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5802 - auc: 0.7290 - loss: 0.6143 - precision: 0.3951 - recall: 0.8155
Epoch 2/60
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7807 - auc: 0.7968 - loss: 0.4786 - precision: 0.6028 - recall: 0.7030
Epoch 3/60
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7892 - auc: 0.7994 - loss: 0.4745 - precision: 0.6298 - recall: 0.6620
Epoch 4/60
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7930 - auc: 0.8056 - loss: 0.4688 - precision: 0.6328 - recall: 0.6709
Epoch 5/60
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7929 - auc: 0.8054 - loss: 0.4695 - precision: 0.6360 - recall: 0.6677
Epoch 6/60
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7914 - auc: 0.8017 - loss: 0.4726 - precision: 0.6334 - recall: 0.6624
Epoch 7/60
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7918 - auc: 0.8021 - loss: 0.4717 - precision: 0.6337 - recall: 0.6586
Epoch 8/60
800/800 ━━━━━━━━━━━━━━━━━━━━ 4

In [69]:
# Predict and classify entire dataset
features = {feature_name: np.array(normalized_dataset[feature_name]) for feature_name in settings.input_features}

prob_poor = experiment_poor_vs_all.model.predict(features).flatten()
prob_standard = experiment_standard_vs_all.model.predict(features).flatten()
prob_good = experiment_good_vs_all.model.predict(features).flatten()

# Store raw probabilities
predictions_df = pd.DataFrame({
    'Poor_Prob': prob_poor,
    'Standard_Prob': prob_standard,
    'Good_Prob': prob_good
})

# Assign final classification based on highest probability with a confidence threshold
threshold = 0.5
predictions_df['Predicted_Credit_Score'] = predictions_df[['Poor_Prob', 'Standard_Prob', 'Good_Prob']].apply(
    lambda x: x.idxmax() if x.max() >= threshold else 'Unknown',  # Default to Standard if no strong prediction
    axis=1
)

# Convert column names to class labels
predictions_df['Predicted_Credit_Score'] = predictions_df['Predicted_Credit_Score'].replace({
    'Poor_Prob': 'Poor',
    'Standard_Prob': 'Standard',
    'Good_Prob': 'Good'
})

# Calculate accuracy
true_labels = dataset['Credit_Score'].values
predicted_labels = predictions_df['Predicted_Credit_Score'].values
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Model Accuracy: {accuracy:.4f}")

# Print classification report
print(classification_report(true_labels, predicted_labels))


3124/3124 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
3124/3124 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
3124/3124 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
Model Accuracy: 0.5481


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

        Good       0.63      0.27      0.38     17823
        Poor       0.69      0.48      0.56     28988
    Standard       0.66      0.68      0.67     53149
     Unknown       0.00      0.00      0.00         0

    accuracy                           0.55     99960
   macro avg       0.49      0.36      0.40     99960
weighted avg       0.66      0.55      0.59     99960



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
